In [1]:
import pandas as pd

# data = pd.read_csv("C:/Users/BERMUDA/Desktop/SuperStoreOrders.csv")
data = pd.read_csv('https://github.com/LMSharma/CodroidHub/raw/master/Dataset/SuperStoreOrders.csv')
# data = pd.read_excel("https://community.tableau.com/servlet/JiveServlet/downloadBody/1236-102-2-15278/Sample%20-%20Superstore.xls")

data['Order Date'] = data['Order Date'].apply(lambda x : pd.datetime.strptime(x,'%d/%m/%Y') )

In [2]:
newDate = pd.datetime.strptime('01-01-2018','%d-%m-%Y')

RFM = data.groupby(by=['Customer ID'],as_index=False).agg({
                                    'Order Date':lambda x : (newDate- x.max()).days,
                                    'Order ID' : lambda x : x.count(),
                                    'Sales': lambda x : x.sum()
                                    }).rename(columns={'Order Date':'Recency','Order ID':'Frequency','Sales':'Monetary'})

In [3]:
RFM.shape

(793, 4)

More recent means better R-Score

In [4]:
rfm_quantiles = RFM.quantile(q=[0.25,0.50,0.75])

rfm_quantiles.head()

,Recency,Frequency,Monetary
0.25,32.0,8.0,1146.050
0.50,77.0,12.0,2256.394
0.75,185.0,16.0,3785.276


In [5]:
# lower the receny number means customer has visited recenlty ==> better RScore.
def rscore(x):
    if x < rfm_quantiles['Recency'][0.25]:
        return 1
    elif x < rfm_quantiles['Recency'][0.50]:
        return 2
    else:
        return 3
    
# greater the frequency and monetray amount better will be the F and M score

def FnMScore(x,param):
    if x < rfm_quantiles[param][0.25]:
        return 3
    elif x < rfm_quantiles[param][0.50]:
        return 2
    else:
        return 1 

In [6]:
RFM['R'] = RFM['Recency'].apply(func=rscore)
RFM['F'] = RFM['Frequency'].apply(func=FnMScore,args=('Frequency',))
RFM['M'] = RFM['Monetary'].apply(func=FnMScore,args=('Monetary',))


RFM['rfmScore'] = RFM.apply(lambda row : row['R']+row['F']+row['M'] , axis=1)

In [7]:
RFM['rfmScore'].unique()

array([6, 5, 4, 9, 7, 3, 8], dtype=int64)

In [8]:
loyaltyCard = ['Platinum','Gold','Silver','Bronze']

RFM['tag']=pd.qcut(RFM['rfmScore'],q=4,labels=loyaltyCard)

In [9]:
RFM.groupby(by=['tag']).count()

,Customer ID,Recency,Frequency,Monetary,R,F,M,rfmScore
tag,,,,,,,,
Platinum,221,221,221,221,221,221,221,221
Gold,185,185,185,185,185,185,185,185
Silver,226,226,226,226,226,226,226,226
Bronze,161,161,161,161,161,161,161,161


In [10]:
RFM.head()

,Customer ID,Recency,Frequency,Monetary,R,F,M,rfmScore,tag
0,AA-10315,186,11,5563.560,3,2,1,6,Silver
1,AA-10375,21,15,1056.390,1,1,3,5,Gold
2,AA-10480,261,12,1790.512,3,1,2,6,Silver
3,AA-10645,57,18,5086.935,2,1,1,4,Platinum
4,AB-10015,417,6,886.156,3,3,3,9,Bronze
